<a href="https://colab.research.google.com/github/ucfilho/ANN/blob/master/06_ANN_build_PSD_set_09_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import random
import numpy as np
import cv2
import zipfile
from random import randint
from PIL import Image
import re
from sklearn.model_selection import train_test_split
#import scikit-image
import skimage
import pandas as pd
import sklearn
import matplotlib.pyplot as plt

In [2]:
!pip install mahotas

In [3]:
# Import the 'transform' module from 'skimage'
from skimage import transform 

In [4]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

In [5]:
# 03_ANN_NEW_DATA... only the grains in 882 are used for training 
#                    the ANN and segmented images are used to 
#                    train no-grain

# 03_ANN_NEW_DATA... only the grains in 882 are used for training the ANN and segmented images are used to train no-grain

In [6]:
!git clone https://github.com/marquesgabi/Doutorado
%cd Doutorado
Transfere='FotosTreino882_and_Segm.zip'
file_name = zipfile.ZipFile(Transfere, 'r')
file_name.extractall()

fatal: destination path 'Doutorado' already exists and is not an empty directory.
/content/Doutorado


# First step: get the segmented file (photos stored in csv file)

In [7]:
labels = [] #name files

with zipfile.ZipFile(Transfere, "r") as f:
  for f in f.namelist():
    labels.append(f)

In [8]:
Num=len(labels)
df=pd.read_csv(labels[0])
for i in range(1,Num):
  df_new=pd.read_csv(labels[i])
  df_new = df_new[~df_new['Type'].isin(['G'])] # drop grain row which is not in 882
  frames = [df, df_new]
  df= pd.concat(frames, ignore_index=True)

y_valor=df['Type']

In [9]:
#print(df.head())

In [10]:
print(df)

      Unnamed: 0 Type          0  ...        782        783  Width
0              0    G    0.00000  ...    0.68631    0.68769    NaN
1              1    G    0.00000  ...    0.49262    0.50909    NaN
2              2    G    0.45321  ...    0.54919    0.55439    NaN
3              3    G    0.75367  ...    0.48524    0.61876    NaN
4              4    G    0.94521  ...    0.98108    0.95838    NaN
...          ...  ...        ...  ...        ...        ...    ...
1627          45    Z  146.27896  ...  177.99518  193.41644  158.0
1628          46    Z  189.54553  ...  174.20529  186.79413  178.0
1629          47    Z   77.16447  ...   68.11906   61.99939  121.0
1630          48    Z   93.59823  ...  157.16385  144.35010  186.0
1631          49    Z  110.17294  ...  128.05782   96.93800  199.0

[1632 rows x 787 columns]


In [11]:
quantidade= df.groupby('Type').size()
print(quantidade)

Type
B    180
G    245
I    180
N    115
V    162
Z    750
dtype: int64


In [12]:
df_G = df[df["Type"] == "G"] 
Cut=['Unnamed: 0','Type','Width']
FotosG= df_G.drop(Cut,axis=1)
#print(FotosG)

In [13]:
Size=28
img_G=[]
# FotosZ= np.array(FotosZ)

Num,cols=FotosG.shape
for i in range(Num):
  data=np.array(FotosG.iloc[i]).reshape(Size,Size)
  img = Image.fromarray(data.astype('uint8'), mode='L')
  img=np.float32(img)
  img28=cv2.resize(img,(Size,Size), interpolation = cv2.INTER_AREA)
  img_G.append(img28)

In [14]:
df_Z = df[df["Type"] == "Z"] 
Cut=['Unnamed: 0','Type','Width']
FotosZ= df_Z.drop(Cut,axis=1)
print(FotosZ)

              0          1          2  ...        781        782        783
882    99.66225  102.53902  105.61966  ...    0.00000    0.00000    0.00000
883    76.92050   71.22499   74.38184  ...   74.69497   74.31098   71.98919
884    93.92000   91.24000   73.32000  ...    0.00000    0.00000    0.00000
885    73.05943   80.09882   82.46129  ...  100.20395  109.51174  120.26761
886   122.86490  123.56383  124.44763  ...  100.99214   96.51134   90.73105
...         ...        ...        ...  ...        ...        ...        ...
1627  146.27896  149.26598  153.09581  ...  161.62216  177.99518  193.41644
1628  189.54553  211.03372  230.51788  ...  167.19521  174.20529  186.79413
1629   77.16447   76.29322   74.31848  ...   69.47470   68.11906   61.99939
1630   93.59823   93.16210   92.73212  ...  159.58344  157.16385  144.35010
1631  110.17294  121.77184  133.03445  ...  134.67851  128.05782   96.93800

[750 rows x 784 columns]


In [15]:
# We'll choose which is grain and withdraw from 750 segmented photos

In [16]:
Size=28
img_Z=[]
# FotosZ= np.array(FotosZ)

Num,cols=FotosZ.shape
for i in range(Num):
  data=np.array(FotosZ.iloc[i]).reshape(Size,Size)
  img = Image.fromarray(data.astype('uint8'), mode='L')
  img=np.float32(img)
  img28=cv2.resize(img,(Size,Size), interpolation = cv2.INTER_AREA)
  img_Z.append(img28)


In [17]:
GRAO=[0,146,149,166,217,222,223,257,268,286,455,482,538,612,644,647,651,677] # 0 ate 749
GRAO=np.array(GRAO)
Ind=FotosZ.index
FotosNG=FotosZ.copy()
for i in GRAO:
  #print(i)
  FotosNG=FotosNG.drop(Ind[i])

In [18]:
PERCENT=245.0/(len(FotosNG.index))
FotosNG=FotosNG.sample(frac=PERCENT, replace=True)
# print(FotosNG)

In [19]:
rows,col=FotosG.shape
y_total=[] # grao-->zero, nao grao-->1
for i in range(rows):
  y_total.append(0) #  # grao-->zero
for i in range(rows,(2*rows)):
  y_total.append(1) #  # nao grao-->zero


In [20]:
frames = [FotosG,FotosNG]
result = pd.concat(frames)

In [21]:
#Define data train and data test

W_train, W_test, yw_train, yw_test = train_test_split(np.array(result), np.array(y_total), 
                                                    test_size=0.30, shuffle=True, 
                                                    random_state=42)

In [22]:
print(len(y_total))
print(len(yw_test))
print(len(yw_train))
A=490*0.3
print(A)

490
147
343
147.0


In [23]:
train_images=W_train #imagens utilizadas para o treino
train_labels=yw_train # resposta esperada para o treino
test_images=W_test
test_labels=yw_test

In [24]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10)
])

In [25]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [26]:
# GRAIN use crop photos other cases segmented
model.fit(train_images, train_labels, epochs=200) 

Epoch 1/200
11/11 [==============================] - 0s 2ms/step - loss: 17.7813 - accuracy: 0.4402 
Epoch 2/200
11/11 [==============================] - 0s 3ms/step - loss: 1.2416 - accuracy: 0.5073
Epoch 3/200
11/11 [==============================] - 0s 2ms/step - loss: 1.0854 - accuracy: 0.4956
Epoch 4/200
11/11 [==============================] - 0s 2ms/step - loss: 0.7768 - accuracy: 0.5044
Epoch 5/200
11/11 [==============================] - 0s 2ms/step - loss: 1.0748 - accuracy: 0.4956
Epoch 6/200
11/11 [==============================] - 0s 3ms/step - loss: 0.7280 - accuracy: 0.5219
Epoch 7/200
11/11 [==============================] - 0s 3ms/step - loss: 0.5312 - accuracy: 0.5714
Epoch 8/200
11/11 [==============================] - 0s 2ms/step - loss: 0.4045 - accuracy: 0.7347
Epoch 9/200
11/11 [==============================] - 0s 2ms/step - loss: 0.2987 - accuracy: 0.9329
Epoch 10/200
11/11 [==============================] - 0s 2ms/step - loss: 1.3182 - accuracy: 0.6385
Epoch 1

In [27]:
#ANN das imagens
x=np.array(W_test)
logits = model(x, training=False)
prediction = tf.argmax(logits, axis=1, output_type=tf.int32)
#print(prediction)

In [28]:
y_valor=np.copy(yw_test)
data = {'y_Actual': y_valor,
        'y_Predicted': prediction
        }  # este dado esta no formato de dicionario

df = pd.DataFrame(data, columns=['y_Actual','y_Predicted'])
#print (df)

In [29]:
confusion_matrix = pd.crosstab(df['y_Actual'], df['y_Predicted'], rownames=['Actual'], colnames=['Predicted'])
print(confusion_matrix)

Predicted   0   1
Actual           
0          72   0
1           1  74


In [30]:
y_true = df['y_Actual']
y_pred = df['y_Predicted']
print(sklearn.metrics.classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99        72
           1       1.00      0.99      0.99        75

    accuracy                           0.99       147
   macro avg       0.99      0.99      0.99       147
weighted avg       0.99      0.99      0.99       147



# ANN find images to be used for PSD (Particle size distribution) determination

In [31]:
# Mahotas used: marquesgabi_fev_2020/02_Mahotas_fracionado_fev_20_2020.ipynb

In [32]:
!git clone https://github.com/marquesgabi/Doutorado
%cd Doutorado
Transfere='Fotos_Grandes_3cdAmostra.zip'
file_name = zipfile.ZipFile(Transfere, 'r')
file_name.extractall()

fatal: destination path 'Doutorado' already exists and is not an empty directory.
/content/Doutorado/Doutorado


In [33]:
labels = [] #name files

with zipfile.ZipFile(Transfere, "r") as f:
  for f in f.namelist():
    labels.append(f)

In [34]:
print(labels)

['Fotos_Grandes-3cdAmostra/Q6-8-4.jpg', 'Fotos_Grandes-3cdAmostra/Q6-5-3.jpg', 'Fotos_Grandes-3cdAmostra/Q6-7-4.jpg', 'Fotos_Grandes-3cdAmostra/Q6-8-2.jpg', 'Fotos_Grandes-3cdAmostra/Q6-3-2.jpg', 'Fotos_Grandes-3cdAmostra/Q6-7-2.jpg', 'Fotos_Grandes-3cdAmostra/Q6-4-4.jpg', 'Fotos_Grandes-3cdAmostra/Q6-9-5.jpg', 'Fotos_Grandes-3cdAmostra/Q6-2-5.jpg', 'Fotos_Grandes-3cdAmostra/Q6-8-3.jpg', 'Fotos_Grandes-3cdAmostra/Q6-9-3.jpg', 'Fotos_Grandes-3cdAmostra/Q6-1-2.jpg', 'Fotos_Grandes-3cdAmostra/Q6-6-3.jpg', 'Fotos_Grandes-3cdAmostra/Q6-3-4.jpg', 'Fotos_Grandes-3cdAmostra/Q6-1-4.jpg', 'Fotos_Grandes-3cdAmostra/Q6-6-2.jpg', 'Fotos_Grandes-3cdAmostra/Q6-4-3.jpg', 'Fotos_Grandes-3cdAmostra/Q6-7-3.jpg', 'Fotos_Grandes-3cdAmostra/Q6-2-2.jpg', 'Fotos_Grandes-3cdAmostra/Q6-9-2.jpg', 'Fotos_Grandes-3cdAmostra/Q6-1-5.jpg', 'Fotos_Grandes-3cdAmostra/Q6-6-5.jpg', 'Fotos_Grandes-3cdAmostra/Q6-2-1.jpg', 'Fotos_Grandes-3cdAmostra/Q6-5-2.jpg', 'Fotos_Grandes-3cdAmostra/Q6-4-1.jpg', 'Fotos_Grandes-3cdAmostr

In [ ]:
print(gxw)

In [ ]:
# NEED TO BE VERIFIED NEXTS PARTS....

In [35]:
Transfere=labels[0]
'''
file_name = zipfile.ZipFile(Transfere, 'r')
file_name.extractall()
'''

"\nfile_name = zipfile.ZipFile(Transfere, 'r')\nfile_name.extractall()\n"

In [36]:
#Start to use the big image
Size=1200 # tamanho da foto
Sub_Size=int(Size/5) # tamanho do fracionamento
Row_Crop=1/2 # posicao do corte
Crop=int(Size*Row_Crop)

In [37]:
!git clone https://github.com/ucfilho/marquesgabi_fev_2020 #clonar do Github
%cd marquesgabi_fev_2020

fatal: destination path 'marquesgabi_fev_2020' already exists and is not an empty directory.
/content/Doutorado/Doutorado/marquesgabi_fev_2020


In [ ]:
# 

In [38]:
import mahotas.features.texture as mht
import mahotas.features
from skimage import filters
from skimage import exposure
import skimage.feature as sk
from google.colab import files
from numpy import linalg as LA
from scipy import stats
from scipy.signal import find_peaks
from scipy.signal import peak_prominences
from scipy.signal import peak_widths
from scipy import integrate
import re
import Go2BlackWhite
import Go2Mahotas

In [39]:
ww,img_name=Go2BlackWhite.BlackWhite(Transfere,Size) #Pegamos a primeira foto Grande
img=ww[0]

FileNotFoundError: ignored

In [ ]:
!ls

In [ ]:
plt.imshow(img, cmap = "gray")

In [ ]:
print(gxw)

In [ ]:

Escolha=['ASM', 'constrast', 'correl', 'variance', 'inv diff mom', 'sum aveg', 
         'sum var', 'sum entropy', 'entropy', 'dif var', 'dif entropy', 
         'IMC1', 'IMC2']

Prop=Escolha[1] # Propriedade a escolher para ter picos, largura de pico, proeminencia,...

In [ ]:
#ANN das imagens
x=np.array(df)
logits = model(x, training=False)
prediction = tf.argmax(logits, axis=1, output_type=tf.int32)
#print(prediction)

In [ ]:
y=np.array(prediction)
Num=len(y)
print(Num)

In [ ]:
print(y)

In [ ]:
print(gxw)

In [ ]:
# select just the grains to show picture
grain=[]
Mahotas_Prop=[]
Todas_Fotos=[] 

img28_all=pd.DataFrame(x)
Width_Grain_2=[]

Size=28 # tamanho da foto
Sub_Size=int(Size/5) # tamanho do fracionamento
Row_Crop=1/2 # posicao do corte
#Row_Crop=1/3 # posicao do corte
Crop=int(Size*Row_Crop)


for i in range(Num):
  if(y[i]==1):
    grain.append(i)
    Width_Grain_2.append(Width_All[i])

cont=0 # 
cols=5
rows=int(len(grain)/cols)+1
Grao_in_All28=[]

for i in range(Num):
  if(y[i]==1):
    Grao_in_All28.append(i)
    cont=cont+1
    plt.subplot(rows,cols,cont) # subplot not allow cont=0
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    Foto=np.array(img28_all.iloc[i]).reshape(28,28)
    plt.imshow(Foto, cmap = "gray")
    plt.xlabel(i)

    Prop_Escolhida=[]
    
    # p_foto=ww[k].reshape(Size,Size)
    p_foto=Foto
    GLCM=[]
    glcm_haralick=[]
    x_ref=[]
    Count=Sub_Size
    p=np.zeros((Sub_Size,Sub_Size))
    j_ref=0
    Cada_foto=[]
    Posicao_X=[]
    Posicao_Y=[]
    for k in range(Size):
      if((k+Sub_Size-1)<Size):
        #print("(k+Sub_Size)=",(k+Sub_Size),"k=",k)
        for i in range(Sub_Size):
          Posicao_X.append(Crop+i)
          for j in range(Sub_Size):
            p[i,j]=p_foto[Crop+i,j+k]
            Posicao_Y.append(j+k)

        WW=np.copy(p) 
        Cada_foto.append(WW.ravel())
        x_ref.append(Count-Sub_Size)
        Count=Count+1
      
        Mahotas =pd.DataFrame(mahotas.features.haralick(p.astype(int)), columns =Escolha)
        Prop_Escolhida.append(Mahotas[Prop].mean())

    Todas_Fotos.append(Prop_Escolhida)

df_mahotas=pd.DataFrame(Todas_Fotos)

plt.subplots_adjust(bottom=0.1, right=1.2, top=2,hspace=0.3, wspace=0.1)

In [ ]:
Features_Total=[]
cont=-1
for i in range(Num):
  if(y[i]==1):
    cont=cont+1
    x_psd=df_mahotas.iloc[cont]
    peaks, rr = find_peaks(x_psd, height=0)

    N_peaks=len(peaks)
    prominences = peak_prominences(x_psd, peaks)
    
    #Area = simps(x, dx=1)
    Area = integrate.simps(x_psd, dx=1)
    if(len(peaks)==0):
      Width_peaks =0
      Width_peaks_max =0
      Width_peaks_min =0
      Media_proem=0    
    else:
      Width_peaks =np.mean(peak_widths(x_psd, peaks, rel_height=0.5))
      Width_peaks_max =np.max(peak_widths(x_psd, peaks, rel_height=0.5))
      Width_peaks_min =np.min(peak_widths(x_psd, peaks, rel_height=0.5))
      Media_proem=np.mean(prominences)
    Median = np.median(x_psd)
    Mode= stats.mode(x_psd)[0]
    Mean=np.mean(x_psd)
    Sd=np.std(x_psd)

    Features=[]
    Features.append(N_peaks)
    Features.append(Media_proem )
    Features.append(Area)
    Features.append(Width_peaks )
    Features.append(Width_peaks_max)
    Features.append(Width_peaks_min)
    Features.append(Median )
    Features.append(Mode[0])
    Features.append(Mean)
    Features.append(Sd)

    Features_Total.append(Features)

Nomes_PSD=['N_peaks','Media_proem','Area','Width_peaks','Width_peaks_max',
                    'Width_peaks_min','Median','Mode','Mean','Sd'] 
  
Features_Total=pd.DataFrame(Features_Total,columns=Nomes_PSD)
    
print(Features_Total)
# Features_total represents properties without considering there is no class 
# Features_total describes all grains found by the ANN

In [ ]:
rows=1;cols=2
k=8
plt.subplot(rows,cols,1)
plt.plot(df_mahotas.iloc[k])
Foto=np.array(img28_all.iloc[Grao_in_All28[k]]).reshape(28,28)
plt.subplot(rows,cols,2)
plt.imshow(Foto, cmap = "gray")

In [ ]:
print(Width_Grain_2)

In [ ]:
Width_Grain=Width_All.iloc[grain]

In [ ]:
Width_Grain=np.array(Width_Grain) # passando de Serie (dataframe 1d) para np.array
print(Width_Grain) 


In [ ]:
print(Width_Grain_2) # lista : tem virgula entre os elementos

# Fifth step create classes

In [ ]:
Width_bounds=[100,200]
N_Class=4
Class=[]
a=Width_bounds[0]
b=Width_bounds[1]
delta_ab=(b-a)/N_Class
for i in range(N_Class-1):
  valor=a+delta_ab*(i+1)
  Class.append(valor)

print(Class)


In [ ]:
Num=len(Width_Grain)
count=[0,0,0,0]
Hist_Width=[]
for i in range(Num):
  if(Width_Grain[i]<Class[0]):
    count[0]=count[0]+1
    Hist_Width.append(0)
  elif(Width_Grain[i]<Class[1]):
    count[1]=count[1]+1
    Hist_Width.append(1)
  elif(Width_Grain[i]<Class[2]):
    count[2]=count[2]+1
    Hist_Width.append(2)
  else:
    count[3]=count[3]+1
    Hist_Width.append(3)

print(count)
N_count=np.copy(count)

Nomes_class=['until_w1','w1-w2','w2-w3','bigger-w3']+Nomes_PSD

In [ ]:
print(Hist_Width)

In [ ]:
print(Features_Total.iloc[0,1])

In [ ]:
rows=len(Width_Grain)
cols=len(Nomes_PSD)
Features_Class=np.zeros((4,cols)) # matrix containing average values stored. Each line is reprenting a different class

for i in range(rows):
  k=Hist_Width[i]
  for j in range(cols):  
    Features_Class[k,j]=Features_Class[k,j]+ Features_Total.iloc[i,j]

print(pd.DataFrame(Features_Class,columns=Nomes_PSD))

In [ ]:
for i in range(4):
  if(N_count[i]==0):
    fator=0
  else:
    fator=1/N_count[i]
  Features_Class[i,:]=Features_Class[i,:]*fator
print(pd.DataFrame(Features_Class,columns=Nomes_PSD))

In [ ]:
print(N_count)